In [ ]:
import math
from random import randint

import glog
import numpy as np
import qiskit

In [ ]:
class Qnote:
  def __init__(self, serial_no, qubits):
    self.serial_no = serial_no
    self.qubits = qubits

  def __str__(self):
    return "serial_no: %u,\nqubits: %s" % (self.serial_no, self.qubits)

In [ ]:
note = Qnote(14520538, qiskit.QuantumCircuit(10))
glog.info("%u, %u" % (note.serial_no, note.qubits.num_qubits))
glog.info(note)

I0707 11:49:18.010632 6052 <ipython-input-16-f8e7ba3d435a>:2] 14520538, 10
I0707 11:49:18.011630 6052 <ipython-input-16-f8e7ba3d435a>:3] serial_no: 14520538,
qubits:      
q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     
q_4: 
     
q_5: 
     
q_6: 
     
q_7: 
     
q_8: 
     
q_9: 
     


In [ ]:
def rand_bool():
  return bool(randint(0, 1))

In [ ]:
class Bank:

  __MIN_LEN = 10
  __MAX_LEN = 15

  def __init__(self):
    self.__note_ledger = {}

  def create_qnote(self):
    len = randint(Bank.__MIN_LEN, Bank.__MAX_LEN)
    bits = [rand_bool() for i in range(len)]
    bases = [rand_bool() for i in range(len)]
    serial_no = None
    while serial_no == None or serial_no in self.__note_ledger:
      serial_no = randint(10 ** Bank.__MIN_LEN, 10 ** Bank.__MAX_LEN)
    self.__note_ledger[serial_no] = (bits, bases)
    return Qnote(serial_no, self.__encode_qubits(bits, bases))

  def __encode_qubits(self, bits, bases):
    qc = None
    if len(bits) == len(bases):
      qc = qiskit.QuantumCircuit(len(bits), len(bits))
      for i in range(len(bases)):
        if bits[i]:
          qc.x(i)
        if bases[i]:
          qc.h(i)
    else:
      glog.error("len of bits and bases must be equal")
    return qc

  def verify_note(self, qnote):
    valid = False
    if qnote.serial_no in self.__note_ledger:
      bits, bases = self.__note_ledger[qnote.serial_no]
      measurement = [bool(int(c)) for c in self.__measure(qnote.qubits, bases)]
      valid = (measurement == bits)
      if not valid:
        qnote.qubits = None # destroy counterfeit qubits
    else:
      glog.warn("Serial Number of given Qnote not found")
    return valid

  def __measure(self, qc, bases):
    measurement = None
    if len(bases) == qc.num_qubits:
      for i in range(qc.num_qubits):
        if bases[i]:
          qc.h(i)
        qc.measure(i, qc.num_qubits - 1 - i)
      backend = qiskit.Aer.get_backend("qasm_simulator")
      sim = qiskit.execute(qc, backend, shots = 1)
      measurement = list(sim.result().get_counts())[0]
    else:
      glog.error("len of bases must equal to qc.num_qubits")
    return measurement


In [ ]:
def create_counterfeit_qnote(legit_qnote):
  num_qubits = legit_qnote.qubits.num_qubits
  qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
  for i in range(num_qubits):
    if rand_bool():
      qc.x(i)
    if rand_bool():
      qc.h(i)
  return Qnote(legit_qnote.serial_no, qc)
  

In [ ]:
GREEN = 32
RED = 31
WHITE = 47

In [ ]:
bank = Bank()
legit = bank.create_qnote()
qnotes = [legit, create_counterfeit_qnote(legit)]

for i in range(len(qnotes)):
  verified = bank.verify_note(qnotes[i])
  correct = bool(i ^ verified)
  glog.info("\33[1;%u;%umValid: %r, Correct: %r", GREEN if correct else RED, WHITE, verified, correct)

I0707 11:49:18.183732 6052 runningpassmanager.py:204] Pass: UnrollCustomDefinitions - 0.00000 (ms)
I0707 11:49:18.184730 6052 basis_translator.py:94] Begin BasisTranslator from source basis {('h', 1), ('x', 1), ('measure', 1)} to target basis {'cu1', 'tdg', 'mcx_gray', 'save_statevector_dict', 'ryy', 'roerror', 'r', 'cx', 'initialize', 'u2', 'cy', 'mcphase', 'u', 'set_stabilizer', 'delay', 't', 'reset', 'barrier', 'save_expval_var', 'csx', 'save_density_matrix', 'save_probabilities', 'ccx', 'mcu3', 'save_probabilities_dict', 'swap', 'save_stabilizer', 'mcrz', 'id', 'measure', 'diagonal', 'snapshot', 'mcu2', 'mcswap', 'mcrx', 'mcsx', 'set_density_matrix', 'save_expval', 'rx', 'mcu1', 's', 'set_statevector', 'rzz', 'mcy', 'rzx', 'h', 'cswap', 'y', 'mcz', 'unitary', 'cp', 'mcry', 'z', 'u3', 'p', 'ry', 'cz', 'save_state', 'u1', 'save_amplitudes_sq', 'rz', 'save_statevector', 'mcx', 'sdg', 'save_amplitudes', 'sx', 'cu2', 'mcr', 'cu3', 'kraus', 'multiplexer', 'rxx', 'x', 'pauli'}.
I0707 11:4

Because of the no cloning theorem and the probability diffence 
that has an almost 100% chance of occurring when someone is evesdropping on the transfer of a decent amount of qubits, the BB84 protocol is basically perfectly secured. Since it is being used for verifying money, it makes it very hard to forge quantum money.


A possible drawback of the Weisner protocol as money is that in order to transfer money, you have to transfer many qubit states. This is not something easily acheived, because you have to prevent the qubits from getting to much noise and decohering. Another drawback could be that the bank has to store the serial number, classical bits, and bases for every qnote.

You could theoretically use trial and error to break the Weisner protocol. If you have the serial number of a valid qnote and the number of qubits, you could create that many qubits in a known state and create a random known bit string of bases, and then after the bank says that it is invalid they would have performed measurements in the correct bases. If you initialized a state to be |0> and it became |1> after measurement or the same with |1> or |+> and |->, you would know for sure that you had the wrong basis for that bit. You would find out its state by continously doing validation with the same base and measuring, and if you got all |0> or all |1> you would know the base is right, and otherwise it is wrong. You could eventually find the correct bases, and then use trial and error for all the possible 2^n bit strings that would be with those bases until the note is verified. The bank could easily prevent this attack by destroying invalid input qubits.